In [1]:
# flatten a multidoc summarization dataset in .jsonl format to a parallel dataset that uses
# the *.sources *.targets format from cnn-dm

# TODO: support shuffling since the cluster items will be sequential by default

# TODO: support formatting with special tokens to indicate document structure (i.e. <SEP> token between Title and Body)


In [4]:
from pathlib import Path
import json
import tqdm

import numpy as np

from transformer_decoding.evaluate import article_to_text

In [5]:
DATADIR = Path('/home/chris/projects/aylien/dynamic-ensembles/data/WCEP')

prefixes = ['train', 'val']
# prefixes = ['val']


shuffle = True
actual_source_only = True


for dataset_prefix in prefixes:
    sources_and_targets = []
    cluster_cnt = 0
    print('loading clusters')
    for cluster in tqdm.tqdm((json.loads(l) for l in open(DATADIR / (dataset_prefix + '.jsonl')))):


        for article in cluster['articles']:
            if actual_source_only:
                # only append one actual source per cluster
                if article['origin'] == 'WCEP':
                    sources_and_targets.append((article_to_text(article), cluster['summary']))
                    break
            else:
                # use all sources per cluster
                sources_and_targets.append((article_to_text(article), cluster['summary']))
        cluster_cnt += 1
    
    output_idxs = np.arange(len(sources_and_targets))
    if shuffle:
        np.random.shuffle(output_idxs)
    
    with open(DATADIR / (dataset_prefix + '.source'), 'w') as srcs, open(DATADIR / (dataset_prefix + '.target'), 'w') as tgts:
        for idx in tqdm.tqdm(output_idxs):
            src = sources_and_targets[idx][0]
            tgt = sources_and_targets[idx][1]
            srcs.write(f'{src}\n')
            tgts.write(f'{tgt}\n')
    print(f'wrote {len(sources_and_targets)} segments from {cluster_cnt} clusters to {srcs.name} and {tgts.name}')
            
        
    

111it [00:00, 1109.16it/s]

loading clusters


8158it [00:08, 928.53it/s] 
100%|██████████| 8158/8158 [00:00<00:00, 143197.27it/s]
84it [00:00, 838.08it/s]

wrote 8158 segments from 8158 clusters to /home/chris/projects/aylien/dynamic-ensembles/data/WCEP/train.source and /home/chris/projects/aylien/dynamic-ensembles/data/WCEP/train.target
loading clusters


1020it [00:01, 737.69it/s]
100%|██████████| 1020/1020 [00:00<00:00, 145893.81it/s]

wrote 1020 segments from 1020 clusters to /home/chris/projects/aylien/dynamic-ensembles/data/WCEP/val.source and /home/chris/projects/aylien/dynamic-ensembles/data/WCEP/val.target
